## **Análise filogenética**

O código  realiza um alinhamento múltiplo de sete sequências, selecionando os melhores alinhamentos de sequências homólogas. Em seguida, processa o cálculo da matriz, utilizando como base o alinhamento obtido com a matriz modelo 'BLOSUM62'. Por fim, concontroi árvores filogenéticas pelos métodos UPGMA e NJ para cada gene em análise.


In [26]:
from Bio import SeqIO, AlignIO, Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import subprocess
import os
import matplotlib.pyplot as plt

def process_gene_with_clustal(gene_name, fasta_file):
    # Criar diretório de saída
    output_dir = "Resultados da Análise Filogenética"
    os.makedirs(output_dir, exist_ok=True)

    # Lê o arquivo FASTA
    sequences = []
    with open(fasta_file) as handle:
        for record in SeqIO.parse(handle, "fasta"):
            sequences.append(record)

    # Verificar se temos pelo menos 7 sequências
    if len(sequences) < 7:
        raise ValueError(f"O arquivo FASTA deve conter pelo menos 7 sequências. Foram encontradas: {len(sequences)}")


    # Salvar as sequências extraídas em um arquivo FASTA
    trimmed_fasta = os.path.join(output_dir, f"{gene_name}_trimmed.fasta")
    SeqIO.write(sequences, trimmed_fasta, "fasta")

    # Caminho para o executável do ClustalW2
    clustal_exe = "clustalw2"  # Substitua pelo caminho correto do executável, se necessário

    # Definir o arquivo de saída do alinhamento
    alignment_file = os.path.join(output_dir, f"{gene_name}_alignment.fasta")

    # Executar o comando ClustalW2 para fazer o alinhamento
    clustal_command = [
        clustal_exe,
        f"/INFILE={trimmed_fasta}",
        f"/OUTFILE={alignment_file}",
        "/OUTPUT=FASTA",
        "/QUIET"
    ]

    print(f"Executando ClustalW2 para o gene {gene_name}...")
    try:
        subprocess.run(clustal_command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Erro ao executar ClustalW2: {e}")
        return

    # Verificar se o alinhamento foi gerado com sucesso
    if not os.path.exists(alignment_file):
        raise FileNotFoundError(f"O alinhamento não foi gerado para o gene {gene_name}.")
    
    # Ler o alinhamento gerado
    alignment = AlignIO.read(alignment_file, "fasta")

    # Garantir que os IDs são únicos e preservar informações importantes
    unique_ids = set()
    for i, record in enumerate(alignment):
        # Extrair o ID original completo
        original_id = record.id
        full_description = record.description
        
        # Processar a descrição para incluir tudo
        full_id = "_".join(full_description.replace(":", "").split())  # Substituir espaços por "_", remover ":"
        
        # Garantir unicidade
        if full_id in unique_ids:
            print(f"ID duplicado encontrado: {full_id}. Renomeando...")
            full_id = f"{full_id}_{i}"  # Adicionar índice se necessário
        
        # Atualizar o ID e descrição do record
        record.id = full_id
        record.description = full_id
        unique_ids.add(full_id)

    # Calcular matriz de distância
    calculator = DistanceCalculator('blosum62')
    try:
        dm = calculator.get_distance(alignment)
        print(f"Matriz de Distância:\n{dm}")
    except ValueError as e:
        print(f"Erro ao calcular a matriz de distância: {e}")
        return

    # Construir árvores
    constructor = DistanceTreeConstructor()
    upgma_tree = constructor.upgma(dm)
    nj_tree = constructor.nj(dm)

    # Remover rótulos 'Inner' dos clades e destacar-los com o género "Homo"
    for tree in [upgma_tree, nj_tree]:
        for clade in tree.find_clades():
            if 'Inner' in (clade.name or ''):
                clade.name = ""
            if 'Homo' in (clade.name or ''):
                clade.color = 'orange'
                clade.name = '*** ' + clade.name + ' ***'  # Marca o clade

    # Guardar árvores
    Phylo.write(upgma_tree, os.path.join(output_dir, f'{gene_name}_upgma_tree.nwk'), 'newick')
    Phylo.write(nj_tree, os.path.join(output_dir, f'{gene_name}_nj_tree.nwk'), 'newick')

    # Processar e desenhar árvores
    for tree, tree_type in [(upgma_tree, 'UPGMA'), (nj_tree, 'NJ')]:
        plt.figure(figsize=(15, 10))
        Phylo.draw(tree, do_show=False)
        plt.title(f'{gene_name} - {tree_type} Tree')
        tree_img = os.path.join(output_dir, f'{gene_name}_{tree_type}_tree.png')
        plt.savefig(tree_img)
        plt.close()

    print(f"Processamento concluído para o gene {gene_name}. Resultados guardados em '{output_dir}'.")


#### **1: Gene ptsP**

In [27]:
process_gene_with_clustal("ptsP", "blast_results/ptsP_blast.fasta")

Executando ClustalW2 para o gene ptsP...
Matriz de Distância:
Faecalibacterium_prausnitzii    0.000000
Faecalibacterium_prausnitzii_1  0.001080    0.000000
Faecalibacterium_prausnitzii_2  0.001080    0.002159    0.000000
Faecalibacterium_prausnitzii_3  0.001439    0.002519    0.002519    0.000000
Faecalibacterium_prausnitzii_4  0.002157    0.003236    0.003236    0.003596    0.000000
Faecalibacterium_sp.    0.001439    0.000360    0.002519    0.002879    0.003596    0.000000
Faecalibacterium_prausnitzii_6  0.001799    0.002879    0.002879    0.003239    0.003955    0.003239    0.000000
    Faecalibacterium_prausnitzii    Faecalibacterium_prausnitzii_1  Faecalibacterium_prausnitzii_2  Faecalibacterium_prausnitzii_3  Faecalibacterium_prausnitzii_4  Faecalibacterium_sp.    Faecalibacterium_prausnitzii_6
Processamento concluído para o gene ptsP. Resultados guardados em 'Resultados da Análise Filogenética'.


<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

#### **2: Gene ButyrylCoA**

In [29]:
process_gene_with_clustal("butyrylCoA", "blast_results/butyrylCoA_blast.fasta")

Executando ClustalW2 para o gene butyrylCoA...
Matriz de Distância:
Faecalibacterium    0.000000
Faecalibacterium_1  0.002124    0.000000
Faecalibacterium_prausnitzii    0.002123    0.004246    0.000000
Faecalibacterium_prausnitzii_M21/2  0.000000    0.002124    0.002123    0.000000
Faecalibacterium_sp.    0.003398    0.001700    0.005520    0.003398    0.000000
Faecalibacterium_prausnitzii_5  0.003398    0.001276    0.005520    0.003398    0.002975    0.000000
Faecalibacterium_prausnitzii_6  0.003398    0.001276    0.005520    0.003398    0.002975    0.002551    0.000000
    Faecalibacterium    Faecalibacterium_1  Faecalibacterium_prausnitzii    Faecalibacterium_prausnitzii_M21/2  Faecalibacterium_sp.    Faecalibacterium_prausnitzii_5  Faecalibacterium_prausnitzii_6
Processamento concluído para o gene butyrylCoA. Resultados guardados em 'Resultados da Análise Filogenética'.


<Figure size 1500x1000 with 0 Axes>

<Figure size 1500x1000 with 0 Axes>

#### **3: Gene MutS Domain I**

In [34]:
process_gene_with_clustal("MutS", "blast_results/MutS_blast.fasta")

Executando ClustalW2 para o gene MutS...


FileNotFoundError: O alinhamento não foi gerado para o gene MutS.